# A1. Python language choices

<div class="alert alert-warning">**Goal of this notebook:**
See how Python fits in with the ideas you learnt in IA _Concepts in programming languages_ and _Object oriented programming_.
</div>

Python development documents are called [_Python Enhancement Proposals_ (PEPs)](https://www.python.org/dev/peps/). Significant changes in the language, or in the standard libraries, are discussed in mailing lists and written up for review as a PEP. PEPs typically suggest several ways a feature might be implemented, and give the reason for choosing one of them. If consensus is not reached to accept the PEP, then the reasons for its rejection are also documented. They are fascinating reading if you are interested in programming language design.

The PEPs are a lot of reading, and so this document is a quick summary of certain features of Python and how they relate to ML and Java.

## A1.1 Dynamic typing

Consider the functions
```
def double_items(xs): 
    return [x*2 for x in xs]
def goodfunc():
    return double_items([1,2,[3,4]]) + double_items("hello world")
def badfunc():
    return double_items(10)
```
Python uses _dynamic typing_, which means that values are tagged with their types during execution and checked only then.
then we won't be told of any errors until `badfunc()` is invoked, even though it's clear before executing it that `badfunc` will fail.

Python programmers are encouraged to use _duck typing_, which means that you should test values for what they can do rather than what they're tagged as. "If it walks like a duck, and it quacks like a duck, then it's a duck". In this example, `double_items(l)` iterates through `xs` and applies `*2` to every element, so it should apply to any `xs` that supports iteration and whose elements all support `*2`. These mean different things: iterating over a list returns its elements while iterating over a string returns its characters; doubling a number is an arithmetical operation, doubling a string or list repeats it.
Python does allow you to test the type of a value with e.g. `if isinstance(x, list): ...`, but programmers are encouraged not to do this. 

Python's philosophy is that library designers are providing a service, and programmers are adults. If a library function uses comparison and addition, and if the end-user programmer invents a new class that supports comparison and addition, then why on earth shouldn't the programmer be allowed to use the library function? (I've found this useful for simulators: I replaced "real-valued timestamp" with "rich timestamp class that supports auditing, listing which events depended on which other events", and I didn't have to change a single line of the simulator body.) Some statically typed languages like Haskell and Scala support this via _dynamic type classes_, but their syntax is rather heavy.

To make duck typing useful, Python has a long list of [special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names) so that you can create custom classes supporting the same operations as numbers, or as lists, or as dictionaries. For example, if you define a new class with the method [`__iter__`](https://docs.python.org/3/reference/datamodel.html#object.__iter__) then your new class can be iterated over just like a list.

### Example: trees
Suppose we want to define a tree whose leaves are integers and whose branches can have an arbitrary number of children. Actually, in Python, there's nothing to define: we can just start using it, using a list to denote a branch node.
```
x = [1,[[2,4,3],9],[5,[6,7],8]]
```
To flatten a list like this we can use duck typing: given a node `n`, try to iterate over its children, and if this fails then the node must be a leaf so just return `[n]`.
```
def flatten(n):
    try:
        return [y for child in n for y in flatten(child)]
    except TypeError as e:
        return [n]
flatten(x)  # returns [1, 2, 4, 3, 9, 5, 6, 7, 8]
```
This would work perfectly well for trees containing arbitrary types &mdash; unless the end-user programmer puts in leaves which are themselves iterable, in which case the duck typing test doesn't work &mdash; unless that is the user's intent all along, to be able to attach new custom sub-branches...

A solution is to define a custom class for branch nodes, and use `isinstance` to test each element to see if it's a branch node. This is not very different to the ML solution, which is to declare nodes to be of type "either leaf or branch" &mdash; except that Python would still allow leaves of arbitrary mixed type.

## A1.2 None, Maybe, Enumeration types

It's often handy for functions to be able to return either a value, or a marker that there is no value. For example, `head(list)` should return a value unless the list is empty in which case there's nothing to return. A common pattern in statically typed languages is to have a datatype that explicitly supports this, for example we'd define `head` to return an enumeration datatype with a constructor function, `None | Some['a]`. This forces everyone who uses `head` to check whether or not the answer is `None`.

In Python, the return type of a function isn't constrained. It's a common convention to return `None` if you have nothing to return, and a value otherwise, and to trust that the person who called you will do the appropriate checks.

Enumeration types are also used for general type restriction, e.g. to limit what can be placed in a list.
When we actually do want to achieve this, Python isn't much help. It does have an [add-on library for enumeration types](https://docs.python.org/3/library/enum.html), but it's a lot of work for little benefit.

One situation where enumeration types are very useful is when working with categorical values in data [Section 3](3.%20Working%20with%20data.ipynb). When working with data, the levels of the enumeration are decided at runtime (by the contents of the data we load in), so a compile-time type annotation is no use; but Python's dynamic typing is also no help in preventing errors.

## A1.3 Functions as values

In Python, as in ML, functions are values. A common pattern is to define functions that return parameterized functions, e.g.
```
def percentile(p):
    def f(xs):
        return xs[int(p*len(xs))]
    return f
```
I can define `median = percentile(.5)` and `lower_quartile = percentile(.25)`, and then apply `median` and `lower_quartile` to lists. The thing returned by `percentile` is a function `f` which also remembers the value `p`; this 'function plus variables it needs' is called a _closure_. In Python, the outer function is called a `decorator`.

In Python web servers, it's common to define decorators for purposes like user authentication, cross-site scripting checks, etc.

## A1.4 Lists, lazy lists and generators<span id="lazy"></span>
Python has lists and lazy lists, which it calls _generators_. The simplest way to define a generator is like a list comprehension, but using `($\cdot$)` rather than `[$\cdot$]`. Consider the following code:
```
def f(x):
    if x == 5:
        raise Exception("Bad value!")
    else:
        return x + 1

xs = range(10)
ys = (f(x) for x in xs)   # doesn't raise exception
zs = [f(x) for x in xs]   # raises exception
```

This code raises an exception on the last line. The preceding line doesn't, because `ys` is a lazy list, not evaluated until you need it. We could for example pull out the first three values of `ys` by
```
[next(ys) for _ in range(3)]
```
and no exception would be raised because we haven't hit the bad value. (Here, `_` is a variable name, like any other. It's a convention to use `_` for loop variables that you don't use for anything.)

Python also has a simple syntax for defining infinite lazy lists, using the `yield`. For example,
```
def fib():
    a,b = 1,1
    while True:
       yield a
       a,b = b, a+b
       
f = fib()
[next(f) for _ in range(20)]  # produces the first 20 Fibonnaci numbers
```
This is like a closure, but richer: the `f` object remembers not only the variables `a` and `b` in its scope, but also the point where it should resume when next `next(f)` is called.

## A1.5 Object-oriented programming

Python is an object-oriented programming language. Every value is an object. You can see the class of an object by calling `x.__class__`. For example,
```
x = 10
x.__class__  # reports int
dir(x)       # gives a list of x's methods and attributes
```
It supports inheritance and multiple inheritance, and static methods, and class variables, and so on. It doesn't support interfaces, because they don't make sense in a duck typing language.

Here's a quick look at a Python object, and at how it might be used for the `flatten` function earlier.
```
class Branch(object):
    def __init__(self, children):
        self.children = children

def flatten(n):
    if isinstance(n, Branch):
        return [y for child in n.children for y in flatten(child)]
    else:
        return [n]

x = Branch([10,Branch([3,2]),"hello"])
flatten(x)

y = Branch([])
y.my_label = "added an attribute"
```
Every method takes as its first argument a variable referring to the current object, `this` in Java.

The last two lines are surprising. You can "monkey patch" an object, after it has been created, to change its attributes or give it new attributes. Like so many language features in Python, this is sometimes tremendously handy, and sometimes the source of infuriating bugs.

Python doesn't support private and protected access modifiers, except by convention. The convention is that attributes and functions whose name beings with an underscore are considered private,
and may be changed in future versions of the library.